# Deploy and Run the Hamiltonian Simulation Function

This interactive guide shows how to upload the Hamiltonian simulation function to Qiskit Serverless and run an example workload.

### Requirements

This guide was developed with the following local package versions:

In [ ]:
from qiskit import __version__ as qiskit_version
from qiskit_ibm_catalog import __version__ as catalog_version

print("qiskit version:", qiskit_version)
print("qiskit catalog version:", catalog_version)

## 1. Authentication

Use `qiskit-ibm-catalog` to authenticate to `QiskitServerless` with your API key (token) and CRN (instance), which you can find on the [IBM Quantum Platform](https://quantum.cloud.ibm.com) dashboard. This will allow you to locally instantiate the serverless client to upload or run the selected function:

```python
from qiskit_ibm_catalog import QiskitServerless
serverless = QiskitServerless(channel="ibm_quantum_platform", token="MY_TOKEN", instance="MY_CRN")
```

You can optionally use `save_account()` to save your credentials in your local environment (see the [Set up your IBM Cloud account](/docs/guides/cloud-setup#cloud-save) guide). Note that this writes your credentials to the same file as [`QiskitRuntimeService.save_account()`](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#save_account):

```python
QiskitServerless.save_account(channel="ibm_quantum_platform", token="MY_TOKEN", instance="MY_CRN")
```

If the account is saved, there is no need to provide the token to authenticate:

In [ ]:
from qiskit_ibm_catalog import QiskitServerless

# Authenticate to the remote cluster
# In this case, using the "ibm_quantum_platform" (IBM Cloud) channel
serverless = QiskitServerless(channel="ibm_quantum_platform")

## 2. Upload the custom function

To upload a Qiskit Function, you must first instantiate a `QiskitFunction` object that defines the function source code. The title will allow you to identify the function once it's in the remote cluster. The main entry point is the file that contains `if __name__ == "__main__"`. If your workflow requires additional source files, you can define a working directory that will be uploaded together with the entry point.

In [ ]:
from qiskit_ibm_catalog import QiskitFunction

template = QiskitFunction(
    title="hamiltonian_simulation_template",
    entrypoint="hamiltonian_sim_entrypoint.py",
    working_dir="./source_files/",  # all files in this directory will be uploaded
)
print(template)

Once the instance is ready, upload it to serverless:

In [ ]:
serverless.upload(template)

To check if the program successfully uploaded, use `serverless.list()`:

In [ ]:
serverless.list()

## 3. Load and run the custom function remotely


The function template has been uploaded, so you can run it remotely with Qiskit Serverless. First, load the template by name:

In [ ]:
template = serverless.load("hamiltonian_simulation_template")
print(template)

Next, run the template with the domain-level inputs for Hamiltonian simulation. This example specifies a 50-qubit XXZ model with random couplings, and an initial state and observable.

In [ ]:
from itertools import chain
import numpy as np
from qiskit.quantum_info import SparsePauliOp

L = 50

# Generate the edge list for this spin-chain
edges = [(i, i + 1) for i in range(L - 1)]
# Generate an edge-coloring so we can make hw-efficient circuits
edges = edges[::2] + edges[1::2]

# Generate random coefficients for our XXZ Hamiltonian
np.random.seed(0)
Js = np.random.rand(L - 1) + 0.5 * np.ones(L - 1)

hamiltonian = SparsePauliOp.from_sparse_list(
    chain.from_iterable(
        [
            [
                ("XX", (i, j), Js[i] / 2),
                ("YY", (i, j), Js[i] / 2),
                ("ZZ", (i, j), Js[i]),
            ]
            for i, j in edges
        ]
    ),
    num_qubits=L,
)
observable = SparsePauliOp.from_sparse_list([("ZZ", (L // 2 - 1, L // 2), 1.0)], num_qubits=L)

In [ ]:
from qiskit import QuantumCircuit

initial_state = QuantumCircuit(L)
for i in range(L):
    if i % 2:
        initial_state.x(i)

In [ ]:
# set up AQC options

aqc_options = {
    "aqc_evolution_time": 0.2,
    "aqc_ansatz_num_trotter_steps": 1,
    "aqc_target_num_trotter_steps": 32,
    "remainder_evolution_time": 0.2,
    "remainder_num_trotter_steps": 4,
    "aqc_max_iterations": 300,
}

job = template.run(
    backend_name="ibm_fez",
    hamiltonian=hamiltonian,
    observable=observable,
    initial_state=initial_state,
    aqc_options=aqc_options,
    estimator_options={},
    dry_run=True,
)
print(job.job_id)

Check the detailed status of the job:

In [ ]:
import time

t0 = time.time()
status = job.status()
if status == "QUEUED":
    print(f"time = {time.time()-t0:.2f}, status = QUEUED")
while True:
    status = job.status()
    if status == "QUEUED":
        continue
    print(f"time = {time.time()-t0:.2f}, status = {status}")
    if status == "DONE" or status == "ERROR":
        break

After the job is running, you can fetch logs created from the `logger.info` outputs. These can provide actionable information about the progress of the Hamiltonian simulation workflow. For example, the value of the objective function during the iterative component of AQC, or the two-qubit depth of the final ISA circuit intended for execution on hardware.

In [ ]:
print(job.logs())

Block the rest of the program until a result is available. After the job is done, you can retrieve the results. These include the domain-level output of Hamiltonian simulation (expectation value) and useful metadata.

In [ ]:
result = job.result()

del result["aqc_final_parameters"]  # the list is too long to conveniently display here
result

Note that the result metadata includes a resource usage summary that allows to better estimate the QPU and CPU time required for each workload (this example runs on dry-run mode so no quantum resources are used).  